In [1]:
import pandas as pd 
import numpy as np 

portfolio = pd.read_excel('./final_exam_data.xlsx',sheet_name='portfolio',index_col=0)
forecasting = pd.read_excel('./final_exam_data.xlsx',sheet_name='forecasting',index_col=0)
fx_carry = pd.read_excel('./final_exam_data.xlsx',sheet_name='fx_carry',index_col=0)
fx_carry.head()

,MXN,SOFR,MXSTR
date,,,
2018-04-03,0.054959,0.000073,0.000298
2018-04-04,0.054852,0.000069,0.000298
2018-04-05,0.055166,0.000069,0.000298
2018-04-06,0.054558,0.000069,0.000298
2018-04-09,0.054871,0.000069,0.000298


## Portfolio

In [38]:
import numpy as np 

def tangency_weights(returns,dropna=True,scale_cov=1, name = 'Tangency'):
    if dropna:
        returns = returns.dropna()

    covmat_full = returns.cov()
    covmat_diag = np.diag(np.diag(covmat_full))
    covmat = scale_cov * covmat_full + (1-scale_cov) * covmat_diag

    weights = np.linalg.solve(covmat,returns.mean())
    weights = weights / weights.sum()
    tangency_weights_df = pd.DataFrame(weights, index=returns.columns, columns=[f'{name} Weights'])

    return tangency_weights_df

def equal_weights(returns): 
    return pd.DataFrame(index = returns.columns, data = 1/len(returns.columns), columns = ['EWP Weights'])

def gmv_weights(returns):
    ones = np.ones(returns.columns.shape)
    cov = returns.cov()* annualization_factor
    cov_inv = np.linalg.inv(cov)
    scaling = 1/(np.transpose(ones) @ cov_inv @ ones)
    gmv_tot = scaling * cov_inv @ ones
    gmv_weights_df = pd.DataFrame(gmv_tot, index=returns.columns, columns=['GMV Weights'])
    return gmv_weights_df

def risk_parity_weights(returns): 
    asset_var = returns.var().to_dict() 
    asset_inv_variance_dict = {asset: 1 / variance for asset, variance in asset_var.items()}
    rp_weights_df = pd.DataFrame.from_dict(asset_inv_variance_dict, orient='index', columns=['RP Weights'])
    return rp_weights_df

def calc_mv_portfolio(mean_rets, cov_matrix, target=None):
    w_tan = tangency_weights(mean_rets)

    if target is None:
        return w_tan

    w_gmv = gmv_weights(mean_rets)
    delta = (target - mean_rets @ w_gmv) / (mean_rets @ w_tan - mean_rets @ w_gmv)
    return delta * w_tan + (1 - delta) * w_gmv

def mv_portfolio_excess_returns(target_ret, ex_ret):
    
    mu_tilde = ex_ret.copy().mean()
    Sigma_adj = ex_ret.copy().cov()
    Sigma_inv = np.linalg.inv(Sigma_adj)
    N = Sigma_adj.shape[0]
    delta_tilde = ((np.ones(N) @ Sigma_inv @ mu_tilde)/(mu_tilde @ Sigma_inv @ mu_tilde)) * target_ret
    tan_wts = tangency_weights(ex_ret)
    omega_star = delta_tilde * tan_wts
    
    return omega_star
    

def mv_portfolio(target_ret, tot_returns):
    
    mu_tan = tot_returns.mean() @ tangency_weights(tot_returns, scale_cov = 1)
    mu_gmv = tot_returns.mean() @ gmv_weights(tot_returns)
    
    delta = (target_ret - mu_gmv[0])/(mu_tan[0] - mu_gmv[0])
    mv_weights = (delta * tangency_weights(tot_returns, scale_cov = 1)).values + ((1-delta)*gmv_weights(tot_returns)).values
    MV = pd.DataFrame(index = tot_returns.columns, data = mv_weights, columns = ['MV Weights'] )
    MV['tangency weights'] =  tangency_weights(tot_returns, scale_cov = 1).values
    MV['GMV weights'] =   gmv_weights(tot_returns).values

    return MV



In [19]:
# normalize returns to his the target

target = 0.0025
tan_weights = tangency_weights(data)
adjustment_factor = target / (data.mean() @ tan_weights.values)
tan_weights *= adjustment_factor
tan_weights

NameError: name 'data' is not defined

# Q1

In [20]:
target = 0.0025
tan_weights = tangency_weights(portfolio)
adjustment_factor = target / (portfolio.mean() @ tan_weights.values)
tan_weights *= adjustment_factor
tan_weights

,Tangency Weights
SPY,0.479969
BTC,0.079311
USO,-0.024120
TLT,-0.022660
IEF,0.093716
IYR,-0.246719
GLD,0.200172


In [21]:
tangency_weights(portfolio)

,Tangency Weights
SPY,0.857595
BTC,0.141710
USO,-0.043098
TLT,-0.040488
IEF,0.167449
IYR,-0.440830
GLD,0.357662


(b)

In [45]:
annualization_factor = 52
gmv_weights(portfolio)

,GMV Weights
SPY,0.092076
BTC,-0.001009
USO,0.002750
TLT,-0.476953
IEF,1.427028
IYR,-0.041140
GLD,-0.002752


In [37]:
    mv_portfolio_excess_returns(0.0025, portfolio)

,Tangency Weights
SPY,0.479969
BTC,0.079311
USO,-0.024120
TLT,-0.022660
IEF,0.093716
IYR,-0.246719
GLD,0.200172


In [30]:
def mv_portfolio(target_ret, tot_returns):
    
    mu_tan = tot_returns.mean() @ tangency_weights(tot_returns, scale_cov = 1)
    mu_gmv = tot_returns.mean() @ gmv_weights(tot_returns)
    
    delta = (target_ret - mu_gmv[0])/(mu_tan[0] - mu_gmv[0])
    mv_weights = (delta * tangency_weights(tot_returns, scale_cov = 1)).values + ((1-delta)*gmv_weights(tot_returns)).values
    MV = pd.DataFrame(index = tot_returns.columns, data = mv_weights, columns = ['MV Weights'] )
    MV['tangency weights'] =  tangency_weights(tot_returns, scale_cov = 1).values
    MV['GMV weights'] =   gmv_weights(tot_returns).values

    return MV
    
mv_portfolio(0.0025, portfolio)


,MV Weights,tangency weights,GMV weights
SPY,0.498982,0.857595,0.092076
BTC,0.074852,0.141710,-0.001009
USO,-0.021620,-0.043098,0.002750
TLT,-0.244953,-0.040488,-0.476953
IEF,0.757509,0.167449,1.427028
IYR,-0.253592,-0.440830,-0.041140
GLD,0.188823,0.357662,-0.002752


(c)
- part 1 involves excess returns which is net of risk free rates while part 2 involves totla returns.
- Excess Returns Approach:

Takes into account the presence of a risk-free asset, which shifts the efficient frontier by incorporating the risk-free rate.
The tangency portfolio includes leverage if the Sharpe ratio is high, assuming the investor can borrow/lend at the risk-free rate.

- Total Returns Approach:

Assumes no risk-free rate is available, or it is ignored for simplicity.
The GMV portfolio minimizes total portfolio risk, regardless of return.
The MV portfolio with a target return focuses solely on absolute returns, without considering excess return over a risk-free rate.
 

In [41]:
# (d)

portfolio_copy = portfolio.drop(columns = ['BTC'])
tangency_weights(portfolio_copy)

,Tangency Weights
SPY,0.975792
USO,-0.034911
TLT,0.012283
IEF,0.071743
IYR,-0.483277
GLD,0.458370


In [84]:
in_sample = portfolio_copy.loc[:'2021',:]
oo_sample = portfolio_copy.loc['2022':'2023':]


In [91]:
target = 0.0025
tan_weights = tangency_weights(in_sample)
adjustment_factor = target / (in_sample.mean() @ tan_weights.values)
tan_weights *= adjustment_factor
is_weights = tan_weights
is_weights


,Tangency Weights
SPY,0.708230
USO,-0.034790
TLT,-0.279598
IEF,1.712950
IYR,-0.252887
GLD,0.023858


In [92]:
oos_returns = oo_sample @ is_weights


In [63]:
def portfolio_metrics_basic(returns: pd.DataFrame, annualization_factor: float) -> pd.DataFrame: 
    '''
    Returns Annualized ER, Vol, SR, 
    '''
    
    returns_df = pd.DataFrame(index = returns.columns) 
    mu = returns.mean() * annualization_factor
    vol = returns.std() * np.sqrt(annualization_factor)
    sr = mu / vol

    
    returns_df['Annnualized Return'] = mu.values 
    returns_df['Annnualized Vol'] = vol.values 
    returns_df['Annnualized SR'] = sr.values
    
    return returns_df


In [107]:
portfolio_metrics_basic(oos_returns, annualization_factor)


NameError: name 'oos_returns' is not defined

In [95]:
ewp = equal_weights(in_sample)
adjustment_factor = target / (in_sample.mean() @ ewp.values)
ewp *= adjustment_factor
oos_weights = ewp
oos_returns = oo_sample @ ewp
portfolio_metrics_basic(oos_returns, annualization_factor)

,Annnualized Return,Annnualized Vol,Annnualized SR
EWP Weights,-0.036569,0.183591,-0.199185


The EWP portfolio performed better with an improved annualized SR and higher expected return, despite experiencing a higher volatility as ell. 

# Q3

In [45]:
forecasting.head()

,USO,Tnote rate,Tnote rate change
date,,,
2009-05-31,0.271394,3.465,0.341
2009-06-30,0.042033,3.523,0.058
2009-07-31,-0.029528,3.501,-0.022
2009-08-31,-0.020647,3.401,-0.100
2009-09-30,0.003883,3.307,-0.094


In [46]:
forecast_copy = forecasting.copy() 
forecast_copy[['Tnote rate shifted', 'Tnote rate change shifted']] = forecasting[['Tnote rate', 'Tnote rate change']].shift()
forecast_copy = forecast_copy.dropna()
import statsmodels.api as sm 
y = forecast_copy['USO'] 
x = forecast_copy[['Tnote rate shifted', 'Tnote rate change shifted']] 
model = sm.OLS(y,sm.add_constant(x)).fit()
model.rsquared

0.02813716766965546

In [58]:
def time_series_regression(portfolio, factors, alpha = True, multiple_factors = False, resid = False, annualization = 12):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    if alpha: 
        rhs = sm.add_constant(factors)
    else: 
        rhs = factors

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] # Note: alpha is not annualized
        if multiple_factors:
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]
            ff_report.loc[portf, factors.columns[1]+ ' beta'] = res.params[2] 
        else:
            print(res.params[1])
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]

            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] =  annualization* portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

In [59]:
USO = forecasting.loc[:,['USO']]
signal = ['Tnote rate','Tnote rate change']
factors = forecasting.loc[:,['Tnote rate','Tnote rate change']].shift(1)
signal_reg = time_series_regression(USO, factors, alpha = True, multiple_factors=True, resid=False)
signal_reg

,alpha_hat,Tnote rate beta,Tnote rate change beta,info_ratio,treynor_ratio,R-squared,Tracking Error
USO,0.02012,-0.009569,0.074061,0.687085,0.440459,0.028137,0.351405


In [60]:
forecasted_rets = (np.array(forecasting.shift(1).loc[:,['Tnote rate','Tnote rate change']]) 
                   @ np.array(signal_reg.loc[:,['Tnote rate beta','Tnote rate change beta']].T))

forecast_ret = (pd.DataFrame(forecasted_rets,columns = ['Forecasted Return'],index= forecasting.index)) 

forecast_ret['Forecasted Return'] = (forecast_ret['Forecasted Return'] + 
                                     float(signal_reg['alpha_hat']))*50 + 0.5

strat = pd.DataFrame(forecast_ret['Forecasted Return'] * forecasting.loc[:,['USO']]['USO'], 
                     columns=forecast_ret.columns, 
                     index=forecast_ret.index).dropna()
strat

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_1454/1586364897.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(signal_reg['alpha_hat']))*50 + 0.5


,Forecasted Return
date,
2009-06-30,0.046697
2009-07-31,-0.001041
2009-08-31,0.005171
2009-09-30,-0.001909
2009-10-31,-0.036695
...,...
2023-07-31,0.053407
2023-08-31,0.003366
2023-09-30,0.003397


In [65]:
portfolio_metrics_basic(strat, 12)

,Annnualized Return,Annnualized Vol,Annnualized SR
Forecasted Return,0.168447,0.3066,0.549401


In [66]:
# amount of USO relative to the returns 
x = forecasting.loc[:,['USO']].iloc[1:]
y = strat
reg = time_series_regression(y,x, alpha = True, multiple_factors=False, resid=False)
reg

0.23541390364566062


,alpha_hat,USO beta,info_ratio,treynor_ratio,R-squared,Tracking Error
Forecasted Return,0.014488,0.235414,0.589537,0.715534,0.074908,0.294893


In [67]:
beta = reg.loc['Forecasted Return', 'USO beta']
hedged = beta * x 
returns = hedged['USO'] + y['Forecasted Return']
mu = returns.mean() * annualization_factor
vol = returns.std() * np.sqrt(annualization_factor)
sr = mu / vol
mu, vol, sr

(3.4238917755922564, 1.554896390047211, 2.2020063828743583)

- Hedge Ratio = 0.235 to hedge
- 3.42%

In [97]:
def OOS_strat(df, factors, start):
    y = df
    X = sm.add_constant(factors)

    forecast_err, null_err,oos_predictions,null_predictions = [], [],[],[]
    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            oos_predictions.append(reg_predict.T)
            null_predictions.append(pd.DataFrame([[reg_predict.index[0]]], columns = ['date'], index = [null_forecast]))
            forecast_err.append(reg_predict.values - actual)
            null_err.append(null_forecast.values - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    predictions_df = pd.concat(oos_predictions).T.drop_duplicates()
    null_predictions_df = pd.concat(null_predictions).T
    
    return ((1 - RSS/TSS),reg,predictions_df,null_predictions_df)

factor = forecasting.loc[:,['Tnote rate','Tnote rate change']].shift(1)
fund_ret = forecasting.loc[factor.index[0]:,['USO']]
OOS_r2, OOS_reg_params, OOS_forecasts, null_predictions_df = OOS_strat(fund_ret,factor, forecasting.loc[:'2017'].shape[0])
OOS_forecasts

date
2018-01-31   -0.007559
2018-02-28    0.002248
2018-03-31   -0.005113
2018-04-30   -0.011960
2018-05-31   -0.002330
                ...   
2023-07-31   -0.006231
2023-08-31   -0.006372
2023-09-30   -0.007037
2023-10-31    0.019210
2023-11-30   -0.002345
Length: 71, dtype: float64

In [98]:
OOS_r2

-0.0044913307399883085

In [103]:
returns = fund_ret.loc['2018-01-31':]
returns['forecasts'] = OOS_forecasts.values
returns.corr()

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_1454/1676219710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns['forecasts'] = OOS_forecasts.values


,USO,forecasts
USO,1.000000,0.031923
forecasts,0.031923,1.000000


In [106]:
def performance_summary(return_data, quantile, annualization_factor):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization_factor)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization_factor))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats[f'VaR ({quantile})'] = return_data.quantile(quantile, axis = 0)
    summary_stats[f'CVaR ({quantile})'] = return_data[return_data <= return_data.quantile(quantile, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats
performance_summary(returns, 0.05, 12)

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
USO,0.074152,0.449894,0.164820,-1.459297,5.763732,-0.163233,-0.343372,-0.554497,0.353556,-0.846005,2018-09-30,2020-04-30,NaT
forecasts,-0.053585,0.060802,-0.881306,-0.496629,1.777322,-0.041627,-0.050289,-0.054034,0.035260,-0.288259,2018-02-28,2023-09-30,NaT


# Q4

In [9]:
fx_carry.columns

Index(['MXN', 'SOFR', 'MXSTR', 'log_MXN', 'log_SOFR', 'log_MXSTR'], dtype='object')

In [33]:
fx_carry['log_MXN'] = np.log(fx_carry['MXN']).fillna(method = 'ffill')
fx_carry['log_SOFR'] = np.log(1+fx_carry['SOFR'])
fx_carry['log_MXSTR'] = np.log(1+fx_carry['MXSTR'])
fx_carry.head(), fx_carry.isna().sum()

(                 MXN      SOFR     MXSTR   log_MXN  log_SOFR  log_MXSTR
 date                                                                   
 2018-04-03  0.054959  0.000073  0.000298 -2.901174  0.000073   0.000298
 2018-04-04  0.054852  0.000069  0.000298 -2.903123  0.000069   0.000298
 2018-04-05  0.055166  0.000069  0.000298 -2.897403  0.000069   0.000298
 2018-04-06  0.054558  0.000069  0.000298 -2.908485  0.000069   0.000298
 2018-04-09  0.054871  0.000069  0.000298 -2.902767  0.000069   0.000298,
 MXN          8
 SOFR         0
 MXSTR        0
 log_MXN      0
 log_SOFR     0
 log_MXSTR    0
 dtype: int64)

In [11]:
fx_carry['log_MXN'].mean()

-2.981326252448035

since these are USD per MXN, to get the log return of holding MXN, -> mxn rate - sofr rate

In [35]:
returns = fx_carry['log_MXN'].diff().shift(-1) + fx_carry['log_MXSTR'] - fx_carry['log_SOFR']
annualization_factor = 252 

mu = returns.mean() * annualization_factor
vol = returns.std() * np.sqrt(annualization_factor)
sr = mu / vol
mu, vol, sr


(0.0635103494622258, 0.1279733380976997, 0.49627797794677825)

In [37]:
(fx_carry['log_MXSTR'] - fx_carry['log_SOFR']).mean()

0.00021860993611677376

- Since this is quoted as USD per MXN, as the expected returns > 0, it shows that the carry trade is expected to be profitable. The investor should be long MXN relative to USD to benefit from the higher return in pesos.
- The interest spread helped on average with a positive spread returned
- USD depreciated relative to MXN over the sample


In [42]:
x = fx_carry['log_SOFR'] - fx_carry['log_MXSTR'] 
y = fx_carry['log_MXN'].diff().shift(-1)
model = sm.OLS(y, sm.add_constant(x), missing = 'drop').fit()
model.rsquared, model.params

(0.00036053074481423586,
 const    0.001136
 0        5.045095
 dtype: float64)

If UIP held, the beta of the interest rate differential would be 1 and alpha would be 0, with a very high r square. In this case, the regression estimates are far from that and since B > 1: FX change > interest rate differential

- If the interest rate differential increased, then we should expect the USD to depreciate relative to MXN
- If Rf in MXN increases, then interest rate differential decreases, forward exchange rate will be lower
- Yes, the estimated forecast impact of interest rates on currency returns would generally be larger over an annual horizon compared to a daily horizon. Over an annual horizon, the impact of interest rate differentials accumulates, compounding the returns. Daily currency movements are often influenced by short-term noise, market sentiment, and other transitory factors, which can obscure the effects of fundamentals like interest rates.